# Prompt Engineering LeoLM

### my slurm settinngs

In [ ]:
#!/bin/bash
#SBATCH --nodes=1
#SBATCH --ntasks=8
#SBATCH --mem=70gb
#SBATCH --gres=gpu:1
#SBATCH --partition=gpu_4_a100
#SBATCH --time=10:00:00

module load devel/python/3.10.5_gnu_12.1
cd /pfs/work7/workspace/scratch/ma_yschiffe-llm

python leolm.py

### Zero Shot

In [ ]:
import pandas as pd
from transformers import pipeline
import torch

df = pd.read_csv('test200.csv')

system_message = """
Wird "[Gruppe1]" im Text als kalt wahrgenommen? 
"""



prompt_format = """
<|im_start|>system
{system_message}<|im_end|>
<|im_start|>user
{prompt}<|im_end|>
<|im_start|>assistant
"""

generator = pipeline(model="LeoLM/leo-hessianai-13b", device="cuda", torch_dtype=torch.float16, trust_remote_code=False)

def generate_response(row):
    formatted_prompt = prompt_format.format(prompt=r['masked'], system_message=system_message)
    answer = generator(formatted_prompt, do_sample=True, top_p=0.95, max_length=8192)
    return answer[0]['generated_text']

df['response'] = df.apply(generate_response, axis=1)

df.to_csv('kalt.csv')

### Zero Shot with Definition

In [ ]:
import pandas as pd
from transformers import pipeline
import torch

df = pd.read_csv('test200.csv')

system_message = """
Wird im Text der Nutzen, die durch die gennante [Gruppe1] für die Gesamtgesellschaft entstehen, explizit genannt, inklusive:
-	Ökonomischer Nutzen, der von der Gruppe generiert wird (Steuer- und Abgabeneinnahmen, Anwerben von Arbeitskräften, Wirtschaftswachstum)
-	Bereitstellung billiger Arbeitskraft
-	Fachkräftemangel im Zielland (muss explizit zusammen mit Zuwanderung genannt werden) – demographischer Wandel: Notwendigkeit des Zuzugs von weiteren Arbeitskräften
-	Es wird EXPLIZIT genannt, dass mehr Arbeitskraft zur Verfügung steht/Wohlstand generiert wird, wenn Ausbildung/Arbeitserlaubnis/Integrationsmaßnahmen vorhanden sind
-	Es wird EXPLIZIT entgangener Nutzen thematisiert, der durch mangelnde Ausbildungsmaßnahmen in Deutschland/Arbeitserlaubnis/Integrationsmaßnahmen entsteht (e.g. „oft fehlt die Anerkennung von Abschlüssen, um an der Wirtschaft teilzunehmen“ „durch unsere restriktive Migrationspolitik fehlen uns Arbeitskräfte“, „durch Diskriminierung wandern nicht genug Fachkräfte zu“)
"""


prompt_format = """
<|im_start|>system
{system_message}<|im_end|>
<|im_start|>user
{prompt}<|im_end|>
<|im_start|>assistant
"""

generator = pipeline(model="LeoLM/leo-hessianai-13b", device="cuda", torch_dtype=torch.float16, trust_remote_code=False)

def generate_response(row):
    formatted_prompt = prompt_format.format(prompt=r['masked'], system_message=system_message)
    answer = generator(formatted_prompt, do_sample=True, top_p=0.95, max_length=8192)
    return answer[0]['generated_text']

df['response'] = df.apply(generate_response, axis=1)

df.to_csv('wirtschaft+.csv')

I tried to give more precise instruction on how to answer but that didn't work.

In [ ]:
system_message = """ 
Wird [Gruppe1] im Text als warm wahrgenommen? Antworte mit 1 für ja oder mit 0 für nein.
[Gruppe1] ist warm, wenn es beschrieben wird als:
-	Vertrauenswürdig/Moralisch: moralisch, vertrauenswürdig, aufrichtig, ehrlich, altruistisch, wohlwollend, warmherzig, loyal, fair, tolerant, gut, tugendhaft, freundlich, richtig, ehrenhaft, unbestechlich, unschuldig, gütlich, authentisch, menschlich, treu, gutmütig, wahrhaftig, kooperativ, nachsichtig, großzügig, versöhnlich, mitfühlend, zuverlässig, verantwortungsbewusst, unvoreingenommen, gut, wohltuend, heroisch, heldenhaft
-	Freundlich/Gesellig: gesellig, freundlich, warm, sympathisch, angenehm, beliebt, kontaktfreudig, sensibel, liebevoll, nicht reserviert, offen, fürsorglich, charmant, hilfreich, verständnisvoll, unterstützend, höflich, sozial, humorvoll, witzig, beliebt, nett, sentimental, zuvorkommend, sanft, angenehm, einladend, gastfreundlich, aufmerksam, graziös
"""

Tried different prompt template

In [ ]:
import pandas as pd
from transformers import pipeline
import torch

df = pd.read_csv('test200.csv')

system_message = """
Wird im Text die Kosten, die durch die gennante [Gruppe1] für die Gesamtgesellschaft entstehen, explizit genannt, inklusive:
-	Kosten von Abschiebungen und Grenzmaßnahmen
-	Kosten bei Hilfszahlungen/Transfers
-	Sozialleistungen im Zielland als Grund für Migration
-	Steigende Arbeitslosigkeit, Arbeitsunwilligkeit oder Arbeitsunfähigkeit der Gruppe genannt (auch: es wird mangelnder Arbeitsanreiz für Gruppe genannt)
-	Verdrängung deutscher Arbeitnehmer durch Gruppe („nehmen unsere Jobs weg“)
-	Kosten der Integration von Gruppenmitgliedern – im Gesundheitssystem, Bildungskosten, Kosten von Sicherheitsmaßnahmen, Bürokratische Belastung, Wohnraum etc. – Es werden Kosten, Kapazitäten, Belastung, Last, Überlastung, Überfüllung, Knappheit, Wohnungsnot genannt, Bedarf an Wohnungen, Mitarbeitern (in Behörden oder zur Versorgung) oder Geld wird explizit genannt
-	Achtung: Spenden zählen nicht, Waffenlieferungen zählen nicht
"""


prompt_format = """
### Instruction
{system_message}

### Input:
{prompt}

### Response
"""

generator = pipeline(model="LeoLM/leo-hessianai-13b", device="cuda", torch_dtype=torch.float16, trust_remote_code=False)

def generate_response(row):
    formatted_prompt = prompt_format.format(prompt=row['masked'], system_message=system_message)
    answer = generator(formatted_prompt, do_sample=True, top_p=0.95, max_length=8192)
    return answer[0]['generated_text']

df['response'] = df.apply(generate_response, axis=1)

df.to_csv('wirtschaft+.csv')

### Few Shot with 2 examples

In [ ]:
import pandas as pd
from transformers import pipeline
import torch

df = pd.read_csv('test200.csv')

message = """
ASYL: Wegen der hohen Zuwanderungszahlen sind die Bundesländer und die Kommunen in Deutschland laut Bayerns Innenminister Joachim Herrmann (CSU) „am Limit“. Finanzielle Hilfe reiche daher nicht aus. Der Bund müsse sich auch um eine Begrenzung der irregulären Migration und eine konsequente Rückführung abgelehnter Asylbewerber kümmern. Und angesichts der angespannten Lage müsse der Bund auch bereits avisierte Aufnahmeprogramme nochmals kritisch hinterfragen. Bereits zugesagt hat die Bundesregierung die Aufnahme von aus Seenot geretteten [Gruppe4], die nach Italien gebracht wurden. Außerdem gibt es ein Programm für besonders schutzbedürftige [Gruppe1].
A: Ja, die wirtschaftlichen Kosten, die durch [Gruppe1] verursacht werden, kommen im Text vor.

Mehr als zwei Jahre nach dem qualvollen Tod von 39 vietnamesischen [Gruppe1] in einem Kühllastwagen in Großbritannien hat ein belgisches Gericht einen Schleuser zu 15 Jahren Haft verurteilt. Der 45-jährige Vietnamese habe eine führende Rolle in dem Fall gespielt, urteilte das Gericht in Brügge. Gegen den Mann wurde zudem eine Geldstrafe in Höhe von knapp einer Million Euro verhängt.
A: Nein, die wirtschaftlichen Kosten, die durch [Gruppe1] verursacht werden, kommen nicht im Text vor.

{prompt}
A: 
"""

generator = pipeline(model="LeoLM/leo-hessianai-13b", device="cuda", torch_dtype=torch.float16, trust_remote_code=False)

def generate_response(row):
    formatted_prompt = message.format(prompt=row['masked'])
    answer = generator(formatted_prompt, do_sample=True, top_p=0.95, max_length=8192)
    return answer[0]['generated_text']

df['response'] = df.apply(generate_response, axis=1)

df.to_csv('fs_wirtschaft-.csv')


### Trying to generate synthetic data 

no results yet since the BW Uni Cluster had an update this week and now the module I am using doesn't work but I will follow up on this approach.

In [ ]:
import pandas as pd
from transformers import pipeline
import torch

message = """
Generiere 10 kurze Zeitungsartikelausschnitte, die Sicherheitsbedrohungen thematisieren, die von Flüchlingen/Migranten/Einwanderer ausgehen. Jeder Ausschnitt soll etwa 3 Sätze umfassen und unterschiedliche Aspekte abdecken.

Hier sind zwei Beispiele:
1. DeSantis gehört wie Trump zum rechten Flügel der Republikanischen Partei. Bei seinen Wahlkampfauftritten spricht er über die „Indoktrinierung“ von Kindern und Jugendlichen an Amerikas Schulen durch die „Verbreitung der Gender-Ideologie“. Im März unterzeichnete er – selbst Vater dreier Kinder – ein Gesetz zum „Schutz der elterlichen Rechte auf die Erziehung“, das Unterricht über sexuelle Orientierung oder Geschlechtsidentität im Kindergarten bis zur dritten Klasse verbietet. Wiederkehrende Themen bei seinen Auftritten sind auch der Anstieg an Kriminalität durch illegale Einwanderer, für den er keine konkreten Belege nennt, sowie die Corona-Politik der US-Regierung, die er für komplett gescheitert hält.
2. Genau diese sei Schuld an der ausufernden Bandenkriminalität, versucht Åkesson, den Schweden zu vermitteln. „Niemand sagt das offen heraus, aber es dreht sich bei den Taten oft um Schießereien unter Migranten-Gangs“, sagt Parteien-Experte Loxbo. „Nirgendwo anders in Westeuropa kommen solche Schießereien so häufig vor, und manche Wähler könnten denken, dass das nach einer radikalen Lösung ruft - und im Ergebnis für die radikale Rechte stimmen.
"""

generator = pipeline(model="LeoLM/leo-hessianai-13b", device="cuda", torch_dtype=torch.float16, trust_remote_code=False)

df = pd.DataFrame()
response = generator(message, do_sample=True, top_p=0.95, max_length=8192)

df['response'] = [response]

df.to_csv('generiert.csv')


### Findings/Limitations
- the LLM sometimes answers in english even though all questions are explicitly asked in german (there’s no instruction for it to answer in a specific language)
- so far it doesn’t stick to the instruction of how to answer
- sometimes it doesn’t give an answer at all and just writes the system message
- since it doesn’t answer in the pattern it is instructed to answer (like 1 or 0), it makes it harder to evaluate the answers. sometimes it answers in 0,4
- all the results so far have led to an F1-Score between 0.39 - 0.5